Karol Janic

kwiecień 2023

zad5.ipynb - Planowanie produkcji w fabryce.

In [1]:
using JuMP
using GLPK

using JSON

In [2]:
dane_wejsciowe = JSON.parse(read("data5.json", String))

maszyny = dane_wejsciowe["maszyny"]
produkty = dane_wejsciowe["produkty"]

Dict{String, Any} with 4 entries:
  "P2" => Dict{String, Any}("popyt"=>100, "cena"=>7, "koszt"=>1, "obrobka"=>Dic…
  "P1" => Dict{String, Any}("popyt"=>400, "cena"=>9, "koszt"=>4, "obrobka"=>Dic…
  "P4" => Dict{String, Any}("popyt"=>500, "cena"=>5, "koszt"=>1, "obrobka"=>Dic…
  "P3" => Dict{String, Any}("popyt"=>150, "cena"=>6, "koszt"=>1, "obrobka"=>Dic…

In [3]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Definicja zmiennych decyzyjnych(ilość produktu w kilogramach do wyprodukowania) oraz ograniczenie ich wartości do liczb nieujemnych:

In [4]:
@variable(model, x[keys(produkty)] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["P2", "P1", "P4", "P3"]
And data, a 4-element Vector{VariableRef}:
 x[P2]
 x[P1]
 x[P4]
 x[P3]

Ograniczenie ilości produkowanego danego produktu do popytu na niego:

In [5]:
@constraint(model, 
    [produkt in keys(produkty)], x[produkt] <= produkty[produkt]["popyt"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["P2", "P1", "P4", "P3"]
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[P2] ≤ 100.0
 x[P1] ≤ 400.0
 x[P4] ≤ 500.0
 x[P3] ≤ 150.0

Ograniczenie możliwości produkcyjnych maszyn:

In [6]:
@constraint(model, 
    [maszyna in keys(maszyny)], sum(x[produkt] * produkty[produkt]["obrobka"][maszyna] for produkt in keys(produkty)) <= maszyny[maszyna]["dostepnosc"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["M3", "M2", "M1"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 4 x[P2] + 6 x[P1] + x[P4] + 3 x[P3] ≤ 3600.0
 6 x[P2] + 10 x[P1] + 2 x[P4] + 5 x[P3] ≤ 3600.0
 3 x[P2] + 5 x[P1] + 4 x[P4] + 4 x[P3] ≤ 3600.0

Funkcja zysku w dolarach(ilość produktu w kilogramach * cena produktu w dolarach za kilogram - ilość produtu w kilogramach * koszt surowców w dolarach za kilogram - liczba przepracowanych godzin przez maszynę / 60 * koszt pracy maszyny w dolarach na minutę):

In [7]:
@objective(model, Max, 
    sum(x[produkt] * produkty[produkt]["cena"] for produkt in keys(produkty)) - sum(x[produkt] * produkty[produkt]["koszt"] for produkt in keys(produkty)) - sum(x[produkt] * produkty[produkt]["obrobka"][maszyna] * maszyny[maszyna]["koszt"] / 60 for produkt in keys(produkty), maszyna in keys(maszyny)))

5.5 x[P2] + 4.2 x[P1] + 3.75 x[P4] + 4.549999999999999 x[P3]

In [8]:
optimize!(model)

In [9]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 3.63250e+03
  Objective bound    : Inf
  Dual objective value : 3.63250e+03

* Work counters
  Solve time (sec)   : 6.58035e-05


In [10]:
JuMP.objective_value(model)

3632.5

In [11]:
for produkt in sort!(collect(keys(produkty)))
    println(produkt, ": ", value(x[produkt]), " kg / tydz")
end

P1: 125.0 kg / tydz
P2: 100.0 kg / tydz
P3: 150.0 kg / tydz
P4: 500.0 kg / tydz
